In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import datetime
import time
import os
from tabulate import tabulate
import requests
from datetime import timedelta
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

# Suppress warnings (if desired)
import warnings
warnings.filterwarnings("ignore")

# -------------------- Initialization --------------------
nltk.download("vader_lexicon")
sia = SentimentIntensityAnalyzer()

# Initialize MT5
if not mt5.initialize():
    print("Failed to initialize MT5")
    exit()
print("Connected to MT5 successfully!\n")

# Set account credentials (update these securely in production)
account = your-ID # mt5 login id
password = "password"  # account password
server = "service-provider"  # seerver name

if mt5.login(account, password, server):
    print("Logged in to MT5 account successfully!\n")
else:
    print("Failed to log in to MT5 account")
    mt5.shutdown()
    exit()

# -------------------- Economic Calendar Functions --------------------
def fetch_tradingview_calendar_chunk(start_date, end_date, countries=None):
    url = 'https://economic-calendar.tradingview.com/events'
    headers = {
        'Origin': 'https://www.tradingview.com',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    payload = {
        'from': f"{start_date}T00:00:00.000Z",
        'to':   f"{end_date}T23:59:59.000Z"
    }
    if countries is not None:
        payload['countries'] = ','.join(countries)
    
    response = requests.get(url, headers=headers, params=payload)
    response.raise_for_status()
    data = response.json()
    if 'result' not in data:
        print("Unexpected JSON structure:", data)
        return pd.DataFrame()
    
    df = pd.DataFrame(data['result'])
    df = df.rename(columns={'date': 'time', 'title': 'event'})
    df = df[['time', 'country', 'event', 'actual', 'forecast', 'previous']].fillna("")
    return df

def fetch_full_calendar(start_date_str, end_date_str, countries=None, chunk_days=30):
    start_date = datetime.datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date   = datetime.datetime.strptime(end_date_str, "%Y-%m-%d")
    current_start = start_date
    all_dfs = []
    while current_start <= end_date:
        current_end = min(current_start + timedelta(days=chunk_days - 1), end_date)
        print(f"Fetching calendar data from {current_start.strftime('%Y-%m-%d')} to {current_end.strftime('%Y-%m-%d')}")
        df_chunk = fetch_tradingview_calendar_chunk(
            current_start.strftime("%Y-%m-%d"),
            current_end.strftime("%Y-%m-%d"),
            countries
        )
        all_dfs.append(df_chunk)
        current_start = current_end + timedelta(days=1)
    return pd.concat(all_dfs, ignore_index=True) if all_dfs else pd.DataFrame()

# -------------------- Market Data Functions --------------------
def fetch_market_data(symbol):
    """
    Fetch the latest 500 bars for the given symbol on a 5-minute timeframe.
    MT5 server time is assumed to be GMT+2; convert to GMT+0 by subtracting 2 hours.
    """
    rates = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_M5, 0, 500)
    if rates is None:
        return pd.DataFrame()
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s') - timedelta(hours=2)
    return df

# -------------------- Impact Analysis --------------------
def analyze_past_events(events, market_data):
    """
    For each past event, compute the volatility (std. dev. of close prices)
    in a 30-minute window around the event time.
    """
    analysis = []
    now = datetime.datetime.utcnow()  # our local time (GMT+0)
    for event in events:
        event_time = datetime.datetime.strptime(event["time"], "%Y-%m-%d %H:%M")
        if event_time >= now:
            continue
        window_start = event_time - timedelta(minutes=30)
        window_end = event_time + timedelta(minutes=30)
        subset = market_data[(market_data['time'] >= window_start) & (market_data['time'] <= window_end)]
        volatility = subset['close'].std() if not subset.empty else ""
        analysis.append({
            "Event": event["event"],
            "Country": event["country"],
            "Time": event["time"],
            "Actual": event["actual"],
            "Forecast": event["forecast"],
            "Previous": event["previous"],
            "Volatility": round(volatility, 4) if volatility != "" else ""
        })
    return analysis

# -------------------- Trade Plan Preparation --------------------
def prepare_trade_plan(upcoming_events, market_data):
    """
    For each upcoming event, prepare a trade plan.
    Calculate support and resistance based on current close price and volatility.
    """
    trade_plans = []
    if market_data.empty:
        return trade_plans
    current_price = market_data['close'].iloc[-1]
    volatility = market_data['close'].pct_change().std()  # as a percentage
    factor = 1.5
    support = current_price * (1 - volatility * factor)
    resistance = current_price * (1 + volatility * factor)
    
    for event in upcoming_events:
        # Dummy prediction: if forecast exists and forecast < previous, then UP; else DOWN.
        if event["forecast"] and event["previous"]:
            predicted_trend = "UP" if float(event["forecast"]) < float(event["previous"]) else "DOWN"
        else:
            predicted_trend = "DOWN"
        probability = f"{round(np.random.uniform(40,70),1)}%"
        logic = f"Based on event details and current volatility, trend expected to be {predicted_trend}."
        trade_plans.append({
            "Event": event["event"],
            "Country": event["country"],
            "Time": event["time"],
            "Prediction": predicted_trend,
            "Support": round(support, 2),
            "Resistance": round(resistance, 2),
            "Probability": probability,
            "Logic": logic
        })
    return trade_plans

# -------------------- Main Execution --------------------
def main():
    symbol = "XAUUSD"
    
    # Clear terminal for fresh output
    os.system("cls" if os.name == "nt" else "clear")
    
    # --- Fetch Economic Calendar for Today ---
    # Here, we fetch the full calendar for today (in UTC, then assume it's GMT+2)
    today = datetime.datetime.utcnow() + timedelta(hours=2)
    start_date_str = today.strftime("%Y-%m-%d")
    end_date_str = start_date_str  # just today
    calendar_df = fetch_full_calendar(start_date_str, end_date_str, countries=None, chunk_days=30)
    
    # Convert time column to readable format if not empty
    if not calendar_df.empty:
        try:
            calendar_df['time'] = pd.to_datetime(calendar_df['time']).dt.strftime("%Y-%m-%d %H:%M")
        except Exception as e:
            print("Time conversion error:", e)
    
    print("\n=== Economic Calendar Events (Today) ===\n")
    if not calendar_df.empty:
        print(tabulate(calendar_df, headers="keys", tablefmt="grid", showindex=False))
    else:
        print("No economic calendar events found.\n")
    
    # --- Fetch Market Data for XAUUSD (Trade Levels) ---
    print(f"\nFetching market data for {symbol} (from start of day until now)...\n")
    market_data = fetch_market_data(symbol)
    if not market_data.empty:
        sample_data = market_data.head(10).copy()
        sample_data['time'] = sample_data['time'].dt.strftime("%Y-%m-%d %H:%M:%S")
        print("Market Data Sample:\n")
        print(tabulate(sample_data, headers="keys", tablefmt="grid"))
    else:
        print("No market data available.\n")
    
    # --- Analyze Impact of Past Events ---
    # For analysis, consider events with time earlier than now (in GMT+2)
    past_events = calendar_df.to_dict("records") if not calendar_df.empty else []
    past_analysis = analyze_past_events(past_events, market_data) if not market_data.empty else []
    print("\nImpact Analysis of Past Events:\n")
    if past_analysis:
        impact_table = [[a["Event"], a["Country"], a["Time"], a["Actual"], a["Forecast"], a["Previous"], a["Volatility"]] for a in past_analysis]
        print(tabulate(impact_table, headers=["Event", "Country", "Time", "Actual", "Forecast", "Previous", "Volatility"], tablefmt="grid"))
    else:
        print("No impact analysis available for past events.\n")
    
    # --- Prepare Trade Plan for Upcoming Events ---
    # Upcoming events: those with time later than now (GMT+2)
    upcoming_events = [e for e in calendar_df.to_dict("records") if datetime.datetime.strptime(e["time"], "%Y-%m-%d %H:%M") >= today]
    trade_plans = prepare_trade_plan(upcoming_events, market_data)
    print("\nTrade Plans for Upcoming Events:\n")
    if trade_plans:
        plan_table = [[p["Event"], p["Country"], p["Time"], p["Prediction"], p["Support"], p["Resistance"], p["Probability"], p["Logic"]] for p in trade_plans]
        print(tabulate(plan_table, headers=["Event", "Country", "Time", "Prediction", "Support", "Resistance", "Probability", "Logic"], tablefmt="grid"))
    else:
        print("No trade plans available for upcoming events.\n")
    
    print("\n=== End of Output ===\n")
    # Pause to review outputs
    time.sleep(300)

if __name__ == "__main__":
    main()
    mt5.shutdown()
